In [5]:
import time
from selenium import webdriver
from selenium.webdriver import ChromeOptions, Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException,TimeoutException

class PostPage:
    translated_text_XPATH = "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/section/div/div/div[1]/div/div/article/div/div/div[3]/div[1]/div/div[3]/div"
    default_text_XPATH = "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/section/div/div/div[1]/div/div/article/div/div/div[3]/div[1]/div/div"
    click_translate_XPATH = "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/section/div/div/div[1]/div/div/article/div/div/div[3]/div[1]/div/div[2]/span"
    usertag_XPATH = "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/section/div/div/div[1]/div/div/article/div/div/div[2]/div[2]/div/div/div[1]/div/div/div[2]/div/div/a/div/span"
    datetime_XPATH = "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/section/div/div/div[1]/div/div/article/div/div/div[3]/div[4]/div/div[1]/div/div/a/time"

class TimelinePage:
    posts_container_XPATH = "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/section/div/div"
    usertag_relative_XPATH = "div/div/article/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[2]/div/div[1]/a/div/span"
    click_target_relative_XPATH = "div/div/article/div/div/div[2]/div[2]/div[2]/div"
    search_XPATH = "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input"
    latest_button_XPATH = "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[2]/a"

def login():
    username = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[autocomplete="username"]')))
    username.send_keys("nouredd31162955")
    username.send_keys(Keys.ENTER)
    password = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[name="password"]')))
    password.send_keys("Vt2drLK?Qh^tfQq")
    password.send_keys(Keys.ENTER)

def search_latest(search):
    search_bar = WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,TimelinePage.search_XPATH)))
    search_bar.send_keys(search)
    search_bar.send_keys(Keys.ENTER)
    latest_button = WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,TimelinePage.latest_button_XPATH)))
    latest_button.click()

def get_posts():
    posts_container = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, TimelinePage.posts_container_XPATH)))
    return posts_container.find_elements(By.XPATH,"div")

def scrap_posts(posts_size = 100):
    pass

def check_post_exists(post_id):
    pass

#Returns index of div in divs
def find_div_by_username(username,posts):
    index = 0
    while index < len(posts):
        post = posts[index]
        usertag = WebDriverWait(post, 10).until(EC.visibility_of_element_located((By.XPATH,TimelinePage.usertag_relative_XPATH)))
        if usertag.text == username:
            return index
        index = index + 1
    return -1
    


In [2]:
options = ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(options)
url = "https://twitter.com/i/flow/login"
driver.get(url)
login()
search_latest("#palestine")

In [6]:
time.sleep(5)
posts_content = []
queue = [get_posts()[0],""]
posts_size = 100
while len(posts_content) < posts_size:
    div = queue[0]
    try:
        click_target = div.find_element(By.XPATH,TimelinePage.click_target_relative_XPATH)
        click_target.location_once_scrolled_into_view
        driver.execute_script("arguments[0].click()",click_target)
        # if click_target.text == "":
        #     queue[0] = find_non_empty_post()
        #     continue
        #now we are in post page
        default_text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, PostPage.default_text_XPATH)))
        try:
            #If not in english a translate button is provided by twitter
            #if non existant => Text is in english => throws error => default behavior in except section.
            translate_target = driver.find_element(By.XPATH,PostPage.click_translate_XPATH)
            print("Non english")
            driver.execute_script("arguments[0].click()",translate_target)
            translated_text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, PostPage.translated_text_XPATH))) 
            text = translated_text.text
        except NoSuchElementException:
            #Text in english
            print("English : ",default_text.text)
            text = default_text.text
        usertag = driver.find_element(By.XPATH,PostPage.usertag_XPATH).text
        time_date = driver.find_element(By.XPATH,PostPage.datetime_XPATH).text
        #check_post_exists(post_id) => add database check
        print({"username":usertag,"time":time_date})
        posts_content.append((usertag,time_date,text))
        driver.back()
        #Back to timeline with same posts
    except:
        print("Ad")
        
    posts = get_posts()
    index = find_div_by_username(posts_content[len(posts_content)-1][0],posts)
    if index == -1:
        queue[0] = posts[0]
    else:
        queue[0] = posts[index+1]
    queue[0].location_once_scrolled_into_view

English :  #Breaking 
Urgent|  The Israeli occupation bombs with artillery shells east of the city of #BeitHanoun in the northern Gaza Strip, and fires lighting bombs into the air.

#Gaza #Palestine 
@qudsn
{'username': '@ferozwala', 'time': '6:24 PM · Mar 29, 2024'}
English :   Yemen "Palestine Is Our Top Cause!"  
An estimated TWO MILLION Yemenis flooded the streets in Sana'a and other cities in a massive show of support for Palestine. #Yemeni #Hezbollah #Palestine #Aleppo
{'username': '@revolutionist47', 'time': '6:24 PM · Mar 29, 2024'}
Non english
{'username': '@LevyThierry', 'time': '6:22 PM · Mar 29, 2024'}
English :  For 6 months they saw no problem , but #InternationalLaw grip for a #genocide has no statute of limitations,so they try now to abandon the sinking ship & show diligence acc to #law_of_war. Too little, too late.They thought that #Palestine was a piece of cake. History ignorants.
{'username': '@rizikaoikonomik', 'time': '6:23 PM · Mar 29, 2024'}
English :   Chicago c

KeyboardInterrupt: 

In [ ]:
print(posts_content)